In [ ]:
import xarray as xr
import copernicusmarine
from datetime import datetime, timedelta


def retrieve_data(lats, longs, parameter, duration, fps):
    """
    Retrieve data from the Copernicus Marine server.

    Parameters
    ----------
    lats : tuple
        (min_lat, max_lat) for the region of interest.
    longs : tuple
        (min_lon, max_lon) for the region of interest.
    parameter : str
        Parameter to retrieve. Options:
        - "chl" : Chlorophyll
        - "sst" : Temperature ("thetao_mean")
        - "sal" : Salinity ("so_mean")
        - "ssh" : Sea Height ("zos_mean")
        - "mlt" : Mixed Layer Thickness ("mlotst_mean")
        - "eke" : Eddy Kinetic Energy ("uo_mean , vo_mean")

    Returns
    -------
    xarray.Dataset
        Dataset containing the requested variable for the specified region and time.
    """
    un="kcastello@ucsd.edu",
    p="2RVrJrqaeHQ@ue_"
    
    # If we want a pretty green chlorophyll plot we need to open a different dataset.
    if parameter == "chl":
        dataset_id = "cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m"
        edt = "2025-08-22T00:00:00"
        depth = 0.4940253794193268
        variables = ["chl"]
    else:
        dataset_id = "cmems_mod_glo_phy-mnstd_my_0.25deg_P1D-m"
        edt = "2023-12-31T00:00:00"
        depth = 0.5057600140571594
        parameter_map = {
            "sst": "thetao_mean",
            "sal": "so_mean",
            "ssh": "zos_mean",
            "mlt": "mlotst_mean",
            "eke": ["uo_mean", "vo_mean"]}
        if parameter not in parameter_map:
            raise ValueError(f"Invalid parameter '{parameter}'. Choose from {list(parameter_map.keys())}")
        variables = parameter_map[parameter]
    if isinstance(variables, str):
        variables = [variables]

    num_frames = int(duration * fps)
    start_date = datetime.fromisoformat(edt) - timedelta(days=num_frames - 1)

    #Reads in xarray help: https://help.marine.copernicus.eu/en/articles/8287609-copernicus-marine-toolbox-api-open-a-dataset-or-read-a-dataframe-remotely
    ds = copernicusmarine.open_dataset(
        dataset_id=dataset_id,
        variables=variables,
        minimum_latitude=lats[0],
        maximum_latitude=lats[1],
        minimum_longitude=longs[0],
        maximum_longitude=longs[1],
        minimum_depth=depth, 
        maximum_depth=depth,
        username=un,
        password=p,
        start_datetime=start_date.strftime("%Y-%m-%dT%H:%M:%S"),
        end_datetime=edt
    )

    if parameter == "eke": 
        u = ds["uo_mean"] 
        v = ds["vo_mean"] 
        eke = 0.5 * (u**2 + v**2) 
        ds = eke.to_dataset(name="eke") 
    ds = ds.drop_vars("depth")
    ds = ds.squeeze("depth", drop=True)
    return ds    

In [ ]:
duration=30
fps = 24
data = retrieve_data((10, 40), (-180, -130), "eke", duration, fps)

In [4]:
#make_gif_from_dataset(data, "eke", filename="current_animation.gif", duration=20)

In [10]:
import hvplot.xarray
import panel as pn
import holoviews as hv

data["eke"].hvplot(groupby="time", widget_location='bottom')

# Create the animation
data["eke"].hvplot(groupby="time", widget_location='bottom', frame_width=750, frame_height=250)
#plot = plot.opts( xaxis=None, yaxis=None, xlabel='', ylabel='', title='', show_legend=False)

#dmap = hv.DynamicMap(plot, kdims=['time'])
#pn.extension()
#pn.panel(dmap, autoplay=True)

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='Time', options={'2022-01-11 00:00:00': np...}, value=np.datetime64('2022-01-11T...)

In [ ]:
import hvplot.xarray
import holoviews as hv
import imageio

hv.extension('bokeh')

# Optional: hide axis labels, ticks, and toolbar
def style_plot(plot):
    return plot.opts(
        frame_width=750,
        frame_height=250,
        xlabel=None,
        ylabel=None,
        xaxis=None,
        yaxis=None,
        toolbar=None,
        show_grid=False,
        title=None
    )

# Create a DynamicMap
dmap = data["eke"].hvplot(groupby="time").map(style_plot)

# Extract all frames
frames = []
for frame in dmap:
    png = hv.render(frame, backend='bokeh').state
    # Save to a temporary PNG buffer
    import io
    from PIL import Image
    buf = io.BytesIO()
    png.screenshot(filename=buf)
    buf.seek(0)
    frames.append(Image.open(buf))

# Save as GIF
frames[0].save(
    'eke_animation.gif',
    format='GIF',
    append_images=frames[1:],
    save_all=True,
    duration=200,  # ms per frame
    loop=0
)